# Markov Decision Processes

First, implement Value iteration to compute the utility 
of the cells in a grid world. Let's first start by setting up the world.

In [1]:
import numpy as np
import sys
%matplotlib notebook
import matplotlib.pyplot as plt


gamma = 1                # The discount factor
backgroundReward = -0.04    # Reward of the non-terminal states
width = 4                   # Width of the grid
height = 3                  # Height of the grid

# We're working in "x,y" coordinates, and address these as such in the 
# numpy matrices. M[x,y] So the matrices are stored with the width rows
# and height columns!

blockedCells = [(1,1)]         # Cells that are not accessible
terminalCells = [(3,0),(3,1)]  # Terminal cells...
terminalRewards = [1,-1]       # ... and the corresponding rewards

# Initialise a matrix containing the rewards:
reward=0.
def initRewards(backgroundReward):
    global reward
    reward = np.zeros((width,height)) + backgroundReward
    for c,r in zip(terminalCells,terminalRewards):
        reward[c]=r
    for c in blockedCells:
        reward[c]=0 # Set the reward in the blocked cell to zero, for later convenience
initRewards(backgroundReward)
print(reward.T) # Print the transposed matrix for correct visualisation

[[-0.04 -0.04 -0.04  1.  ]
 [-0.04  0.   -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]


For convenience, implement some functions to evaluate if 
given coordinates correspond to a valid cell, what the coordinates
would be if we performed a certain action (up, left, down, right) with
no stochasticity but taking the walls into account, and a pretty-printing function for a policy.

In [2]:

def valid(x,y):
    """Return whether (x,y) is a valid grid position"""
    return x >= 0 and x < width and y >= 0 and y < height and (x,y) not in blockedCells

def doAction(x,y,a):
    """What is the result of action a in cell (x,y) in a deterministic world"""
    if (x,y) in terminalCells or (x,y) in blockedCells:
        return blockedCells[0]  # Where the reward is zero
    cur = (x,y)

    
    if a == 0: # north
        y -= 1
    elif a == 1: # east
        x += 1
    elif a == 2: # south
        y += 1;
    elif a == 3: # west
        x -= 1

    if valid(x,y):
        return (x,y)

    return cur

def printPolicy(policy):
    """Print a policy table, that is, an action for each possible state,
    in human-readable format"""
    actions = [
        "Up    ",
        "Right ",
        "Down  ",
        "Left  ",
        "      ", # -1, no action
    ]
    for x in range(policy.shape[0]):
        for y in range(policy.shape[1]):
            print(actions[policy[x,y]],end="")
        print()
    print()

## Question 1

Recall that value iteration updates the utilities as follows:

$$
U_i(s) \leftarrow R(s) + \gamma \max_{a\in A(s)} \sum_{s'} P(s'\mid s,a) \, U_{i-1}(s')
$$

Implement a function, which we will call ```bestValue(x,y,utility)``` which takes as arguments
- the coordinates x,y of the current cell
- utility, the table of the "old" estimates of the utility values, $U_{i-1}$

and returns
- The action $a$ for which the highest expected value is obtained, and
- the expected value of that action: $\sum_{s'} P(s'\mid s,a) \, U_{i-1}(s')$

For cells where no action is possible (terminal states), we return no action (index -1) and zero value. We use the transition probabilities from the book ($0.8$ for the intended actions, and $0.1$ for the two perpendicular motions)

In [3]:
#print(reward[0,0])

def bestValue(x,y,utility):
    """Return the best action we can take in cell x,y, and its value"""
    bestUtility = float('-inf')
    currentUtility = float('-inf')
    action = 0
    if (x,y) in terminalCells or (x,y) in blockedCells:
        return -1,0.0
    else:
        for i in range(4):
            currentUtility = 0.8*utility[doAction(x,y,i)] + 0.1*utility[doAction(x,y,((i-1) % 4))] + 0.1*utility[doAction(x,y,((i+1) % 4))]
            if (currentUtility > bestUtility):
                bestUtility = currentUtility
                action = i
    #return (action, round(bestUtility, 3))
    return action,bestUtility


\## Question 2

Using the above function, implement a function ```valueIteration```, which implements a single iteration of the value iteration algorithm. It takes the current utility table as input, and returns:
- the new estimate of the utility table
- the corresponding policy, that is, the table of best actions for each grid location

In [4]:
def valueIteration(utility):
    policy = np.zeros((width,height), dtype=int)
    newUtility = utility.copy()
    """Fill in the next utility table ut with the updated 
    values from the previous utility table ut1, and store the 
    best action for each cell in the policy table"""
    # Your code comes here
    
    for i in range(width):
        for y in range(height):
            newUtility[i][y]=(gamma*bestValue(i,y,utility)[1])+reward[i][y]
            policy[i][y]=(bestValue(i,y,utility)[0])
    return newUtility,policy
#printPolicy(valueIteration(utility)[1])

## Question 3

The following code uses above implementated functions to implement value iteration. For each iteration, it stores the largest change in utility in the "delta" list, and stops if the list hasn't changed yet.

- How fast does the algorithm converge for different values of $\gamma$? (For example, $\gamma=.1$? What is it for $\gamma=.9$, $\gamma=1$)


- Is the utility the same for these different values of $\gamma$?


- Is the resulting policy the same for these different values of $\gamma$?


In [5]:
def iterate():
    utility = np.zeros((width,height))
    delta = []
    while len(delta) < 5 or delta[-2]-delta[-1] > 0.0:
        nextut, policy = valueIteration(utility)
        delta.append((nextut-utility).max())
        #print(nextut.T)
        #printPolicy(policy.T)
        utility = nextut.copy()
    print("Number of iterations for gamma = ",gamma," :",len(delta))
    print(backgroundReward)
    return delta, utility.T, policy.T

print(iterate()[1],'\n',iterate()[2],'\n')
gamma = 0.9
print(iterate()[1],'\n',iterate()[2],'\n')
gamma = 0.1
print(iterate()[1],'\n',iterate()[2])

# As gamma decreases the number iterations also decrease

# The utility changes since the number of iteration has decreased resulting in values smaller than the ones
#with gamma 1. When gamma is 0.1 the utility of most cells is negative

# The policies are also different. As gamma decreases the agent chooses shorter paths and when gamma is 0.1 it sometimes chooses 
# an invalid cell because the utility of that cell is 0.

gamma = 1

Number of iterations for gamma =  1  : 60
-0.04
Number of iterations for gamma =  1  : 60
-0.04
[[ 0.81155822  0.86780822  0.91780822  1.        ]
 [ 0.76155822  0.          0.66027397 -1.        ]
 [ 0.70530822  0.65530822  0.61141553  0.38792491]] 
 [[ 1  1  1 -1]
 [ 0 -1  0 -1]
 [ 0  3  3  3]] 

Number of iterations for gamma =  0.9  : 49
-0.04
Number of iterations for gamma =  0.9  : 49
-0.04
[[ 0.5094156   0.64958636  0.79536224  1.        ]
 [ 0.39851125  0.          0.48644046 -1.        ]
 [ 0.29646654  0.25396055  0.3447884   0.12994247]] 
 [[ 1  1  1 -1]
 [ 0 -1  0 -1]
 [ 0  1  0  3]] 

Number of iterations for gamma =  0.1  : 19
-0.04
Number of iterations for gamma =  0.1  : 19
-0.04
[[-0.04388718 -0.03755393  0.03996438  1.        ]
 [-0.04439895  0.         -0.04352616 -1.        ]
 [-0.04444071 -0.04443844 -0.04437091 -0.04444364]] 
 [[ 1  1  1 -1]
 [ 0 -1  3 -1]
 [ 0  1  0  2]]


## Question 4

Print out the policies you get with the following rewards for non-terminal states (with $\gamma=1$):
- -2
- -1
- -0.4
- -0.01
- 1

In [6]:
backgroundReward = -2
initRewards(backgroundReward)
print(iterate()[2])

Number of iterations for gamma =  1  : 5
-2
[[ 1  1  1 -1]
 [ 0 -1  1 -1]
 [ 1  1  1  0]]


In [7]:
backgroundReward = -1
initRewards(backgroundReward)
print(iterate()[2])

Number of iterations for gamma =  1  : 47
-1
[[ 1  1  1 -1]
 [ 0 -1  0 -1]
 [ 1  1  0  0]]


In [8]:
backgroundReward = -0.4
initRewards(backgroundReward)
print(iterate()[2])

Number of iterations for gamma =  1  : 54
-0.4
[[ 1  1  1 -1]
 [ 0 -1  0 -1]
 [ 0  1  0  3]]


In [9]:
backgroundReward = -0.01
initRewards(backgroundReward)
print(iterate()[2])

Number of iterations for gamma =  1  : 309
-0.01
[[ 1  1  1 -1]
 [ 0 -1  3 -1]
 [ 0  3  3  2]]


In [10]:
backgroundReward = 1
initRewards(backgroundReward)
print(iterate()[2])

Number of iterations for gamma =  1  : 5
1
[[ 0  0  3 -1]
 [ 0 -1  3 -1]
 [ 0  0  0  2]]
